# MAC0318 - Teoria de controle - Notebook 1

*Pré-requisitos*: 
- **Software**: Para rodar este notebook corretamente, você deve ter o Python na versão 3.9 ou mais recente e a biblioteca NumPy instalada. Antes de começar, assegure-se de que essas condições são satisfeitas.
- **Conhecimento**: Esta atividade assume familiaridade com o uso de notebooks Jupyter e a manipulação de matrizes usando a bilbioteca NumPy. Recomendamos seguir algum tutorial da internet antes de realizar essa atividade, se esse não for o seu caso.

<span style="color:blue">Se você está rodando o notebook em sala de aula, você deve ativar o ambiente `duckietown` do conda antes de abrir o jupyter notebook e escolher o kernel `duckietown`, para que os requerimentos sejam satisfeitos.</red>

## Representação de estado

Nesse tutorial revisamos alguns conceitos relacionados a sistemas referenciais, mudanças de coordenadas e operações vetoriais e matriciais usando o NumPy.

O **estado** descreve matematicamente a configuração do robô em um determinado instante. Decidir o que constitui o estado faz parte da modelagem e projeto do sistema de controle. Podemos por exemplo incluir informações não diretamente mensuráveis, como a localização do robô no ambiente, assim como informações que podem ser inferidas ou estimadas pelo robô, como seu nível de bateria, a quantidade de rotações feitas por cada motor e a temperatura do ambiente. A escolha do estado depende da tarefa que queremos resolver, assim como do domínio de atuação.

Idealmente, gostaríamos que o estado possuisse as seguintes propriedades:

1. *Propriedade de Markov*: O estado futuro independente do passado dado o presente


$$
x_{t+1} = f(x_t, x_{t-1}, \dots, x_0; u_t, \dots, u_0) = f(x_t; u_t) ,
$$
onde $`f`$ é uma função que descreve a evolução do estado do robô em função dos estados anteriores e de outras variáveis (como veremos mais adiante).

2. *Suficência e eficiência estatística*: O estado é um estatística suficiente minimal, ou seja, contém todas e apenas informações relevantes para resolução da tarefa pretendida.
3. *Tratabilidade*: o estado permite calcular eficientemente o comportamento do robô.
4. *Generalibilidade*: o estado é robusto a pequenas mudanças no domínio e na tarefa.

## Pose

O exemplo mais comum de estado em robótica é a *pose* ou *postura* do robô.

<figure style="float:right">
    <div style="text-align:center;">
        <img src="img/robot_kinematics.png" width="380" alt="robot kinematics">
    </div>
</figure>

A **pose** contém a posição do robô no ambiente (relativa a um referencial) assim como sua orientação. Para um robô que atue apenas no solo, como é nosso caso nessa disciplina, ignorando eventuais diferenças de altitude do solo, podemos assumir que a pose $`q`$ é dada pelas coordenadas do centro de referencial local do robô $(x_A,y_A)$ e do ângulo $\theta$ de rotação do referencial em relação a um referencial global fixo:

$$
q = 
\begin{bmatrix}
x & y & \theta
\end{bmatrix}
$$

Outra forma de representar a pose é considerar a **transformação linear homogênea** em relação a uma posição inicial (em relação a algum referencial), formada por uma translação $`t=\begin{bmatrix}
x &
y 
\end{bmatrix}^t`$ seguida por uma rotação $`R`$:

$$
T = 
\begin{bmatrix}
R & t \\
0 & 1
\end{bmatrix}
=
\begin{bmatrix}
\cos(\theta) & -\sin(\theta) & x \\
\sin(\theta) & \cos(\theta) & y \\
0 & 0 & 1
\end{bmatrix}
$$
onde $R$ é a matriz de rotação
$$
R = 
\begin{bmatrix}
\cos(\theta) & -\sin(\theta) \\
\sin(\theta) & \cos(\theta) \\
\end{bmatrix}.
$$

A matriz $`T`$ faz parte do chamado [grupo](https://en.wikipedia.org/wiki/Euclidean_group) (no sentido algébrico) especial euclideano de dimensão 2, denominado $SE(2)$, que representa as simetrias do espaço euclideano que preservam distância entre pontos e orientação, ou seja, rotação e translação.

Uma propriedade importante da matriz de transformação $T$ é que sua inversa observa:
$$
T^{-1} = 
\begin{bmatrix}
R^T & -R^Tt \\
0 & 1
\end{bmatrix} .
$$

In [1]:
# Vamos primeiro rever alguns comandos úteis em numpy
import numpy as np
# Criando um vetor
r = np.array([ 3.0, 4.0 ])
print('r =', r)

r = [3. 4.]


In [2]:
# Criando uma matriz 2x2 (como por exemplo, a matriz de rotação com theta=0.5 rad)
R = np.array([ 
                [np.cos(0.5), -np.sin(0.5)],
                [np.sin(0.5),  np.cos(0.5)]
             ])
print('R =\n', R)
print(R.ndim, R.shape)

R =
 [[ 0.87758256 -0.47942554]
 [ 0.47942554  0.87758256]]
2 (2, 2)


As matrizes de rotação pertencem ao grupo especial ortogonal $`SO(2)`$, que observam a propriedade que suas inversas são dadas por suas transpostas:
$$`
R^{-1} = R^T
`$$

In [4]:
# Inversa da matriz R
Rinv = np.linalg.inv(R)
print('inv(R) =\n', Rinv)
# Transposta da matriz R
Rt = R.T
print('Rt =\n', Rt)
print('Rt ≈ inv(R)? ', np.allclose(Rt, Rinv)) # verdade para matriz em SO(2)

inv(R) =
 [[ 0.87758256  0.47942554]
 [-0.47942554  0.87758256]]
Rt =
 [[ 0.87758256  0.47942554]
 [-0.47942554  0.87758256]]
Rt ≈ inv(R)?  True


In [7]:
# Compondo uma matriz de blocos (matriz de transformação linear homogênea)
T = np.block([
                [R, r.reshape((2,1))], 
                [np.zeros((1, 2)), np.array([1])]
              ])
print('T =\n', T)

T =
 [[ 0.87758256 -0.47942554  3.        ]
 [ 0.47942554  0.87758256  4.        ]
 [ 0.          0.          1.        ]]


In [8]:
# Inversa de matriz em SE(2)
Tinv = np.block([
                 [Rt, -Rt @ r.reshape((2,1))],
                 [np.zeros((1,2)), np.array([1])]
                ])
print('T^-1 =\n', Tinv)

T^-1 =
 [[ 0.87758256  0.47942554 -4.55044984]
 [-0.47942554  0.87758256 -2.07205363]
 [ 0.          0.          1.        ]]


In [11]:
# Vamos verificar que Tinv é realmente a inversa de T:

# Multiplicação de matrizes
I = T @ Tinv # multiplicação de matrizes
print('I =\n', I) 
print('I é matriz identidade?', np.allclose(I,np.identity(3))) 

I =
 [[ 1.00000000e+00 -2.58022041e-17  0.00000000e+00]
 [-2.58022041e-17  1.00000000e+00 -8.88178420e-16]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
I é matriz identidade? True


## Exercício 1

A figura abaixo ilustra a posição do robô (ponto laranja) e sua orientação ($\theta$), dada pelo ângulo entre o eixo $x$ no sentido antihorário.

<figure>
  <div style="text-align:center;">
  <img src="img/pose_exercise.png">
  </div>
</figure>


1. Escreva o vetor $q \in \mathbb{R}^3$ representando a pose do robô no referencial global $W$

In [1]:
theta = np.deg2rad(0) # converte de graus para radianos = (np.pi * 60) / 180.

q_W = np.array([0, 0, theta]) # complete com sua solução

print(q_W)

[0. 0. 0.]


2. Escreva o vetor $q \in \mathbb{R}^3$ representando a pose do robô no referencial local (do robô) $R$

In [2]:
q_R = np.array([0, 0, 0]) # complete com solução

print(q_R)

[0 0 0]


3. Escreva a matriz $T \in SE(2) \subseteq \mathbb{R}^{3 \times 3}$ representando a pose do robô relativa ao referencial global

In [10]:
T = np.array([
    [np.cos(theta), -np.sin(theta), q_g[0]],
    [np.sin(theta),  np.cos(theta), q_g[1]],
    [            0,              0, 1]
])

print(T) # Esperado: [[ 0.5       -0.8660254  2.       ]
         #            [ 0.8660254  0.5        3.       ]
         #            [ 0.         0.         1.       ]]

[[ 0.5       -0.8660254  2.       ]
 [ 0.8660254  0.5        3.       ]
 [ 0.         0.         1.       ]]


# Mudança de referencial

A representação como matriz em $SE(n)$ é conveniente pois permite facilmente que a descrição da pose em um referencial seja reescrita com relação a outro referencial. 

Por exemplo, se sabemos a pose $T_A^O$ no referencial de origem $O$ e sabemos a pose $T_B^A$ no referencial $A$, obtemos a pose $T_B^O$ no referencial de origem como
$$
T_B^O = T_A^O T_B^A .
$$

Outra transformação útil entre referenciais é
$$
T_A^B = (T_B^A)^{-1} .
$$


## Exercício 2

Na figura abaixo, o ponto laranja representa o robô (uma pose) e o ponto azul representa o obstáculo (um ponto). As medidas dos eixos são em metros.

<figure>
  <div style="text-align:center;">
  <img src="img/moving_frame_exercise_1.png">
  </div>
</figure>

Sabendo que a pose do robô no referencial global é dada por
$$
q_r^W = \begin{bmatrix} 2 & 0.4 & 110 \end{bmatrix} 
$$
e que a distância e o ângulo entre o robô e obstáculo são de $`0{,}3`$m e $50$ graus, respectivamente, determine a posição do obstáculo no referencial global.

In [ ]:
# Escreva sua solução
T_O_W = #?

print('T_O^W =\n', T_O_W ) 

In [10]:
# Complete com sua solução (posição do obstáculo no referencial global)
p_O_W = np.array([T_O_W[0,2], T_O_W[1,2]])    
print('p_O^W =', p_O_W) # resultado esperado: [1.71809221, 0.50260604, 160.]

[0. 0.]


## Exercício 3

Na figura abaixo, a pose do robô no referencial global é  $q^W = \begin{bmatrix} 3.5 & -1.2 & 45 \end{bmatrix}$.

<figure>
  <div style="text-align:center;">
  <img src="img/moving_frame_exercise_2.png">
  </div>
</figure>

Determine a posição do obstáculo (ponto azul) na figura abaixo no referencial local do robô.

In [ ]:
p_R_W = np.array([3.5, -1.2]) # posição do robô no referencial global
theta_R_W = np.deg2rad(45)    # ângulo do robô no referencial global

p_O_W = np.array([4.0, -1.0]) # posição do obstáculo no referencial global

# Escreva sua solução


p_O_R = np.array([ T_O_R[0,2], T_O_R[1,2] ])

print('p_O^R =', p_O_R) # resultado esperado [ 0.49497475 -0.21213203]

In [14]:
# Solução alternativa

T_W_R @ np.block([p_O_W, 1])

array([ 0.49497475, -0.21213203,  1.        ])

Pronto! Agora submeta suas soluções no e-disciplina.